In [29]:
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np

df = pd.read_csv('genshin.csv')

data = {
    'character_name': df['character_name'],
    'birthday': df['birthday'],
    'release_date': df['release_date'],
    'character_name': df['character_name'],
    'affiliation': df['affiliation'],
    'ascension_specialty': df['ascension_specialty'],
    'description': df['description'],
    'has_anemo_vision': (df['vision'] == 'Anemo').astype(int),
    'has_geo_vision': (df['vision'] == 'Geo').astype(int),
    'has_dendro_vision': (df['vision'] == 'Dendro').astype(int),
    'has_pyro_vision': (df['vision'] == 'Pyro').astype(int),
    'has_cryo_vision': (df['vision'] == 'Cryo').astype(int),
    'has_hydro_vision': (df['vision'] == 'Hydro').astype(int),
    'has_electro_vision': (df['vision'] == 'Electro').astype(int),
    'has_bow_weapon': (df['weapon_type'] == 'Bow').astype(int),
    'has_sword_weapon': (df['weapon_type'] == 'Sword').astype(int),
    'has_claymore_weapon': (df['weapon_type'] == 'Claymore').astype(int),
    'has_polearm_weapon': (df['weapon_type'] == 'Polearm').astype(int),
    'is_catalyst': (df['weapon_type'] == 'Catalyst').astype(int),
    'is_female': df['model'].isin(['Medium Female', 'Tall Female', 'Short Female']).astype(int),
    'is_male': df['model'].isin(['Medium Male', 'Tall Male']).astype(int),
    'is_mommy': (df['model'] == 'Tall Female').astype(int),
    'is_daddy': (df['model'] == 'Tall Male').astype(int),
    'is_child': (df['model'] == 'Short Female').astype(int),
    'from_Inazuma': (df['region'] == 'Inazuma').astype(int),
    'from_Snezhnaya': (df['region'] == 'Snezhnaya').astype(int),
    'from_Sumeru': (df['region'] == 'Sumeru').astype(int),
    'from_Liyue': (df['region'] == 'Liyue').astype(int),
    'from_Mondstadt': (df['region'] == 'Mondstadt').astype(int),
    'from_Fontaine': (df['region'] == 'Fontaine').astype(int),
    'is_archon': (df['is_archon']).astype(int),
    'has_his_own_banner': (df['limited']).astype(int),
    'is_free': (df['is_free']).astype(int),
    'is_legendary': (df['rarity'] == 5).astype(int),
}

df = pd.DataFrame(data)

features = df.columns[6:]

target = 'character_name'

X = df[features]
y = df[target]

clf = DecisionTreeClassifier()
clf = clf.fit(X, y)

def ask_question(feature):
    formatted_feature = feature.replace('_', ' ')
    return input(f"Does the character have {formatted_feature.lower()}? (yes/no/back): ")


def akinator_game(model, features, characters_df):
    node = 0
    cumulative_confidence = 100.0
    previous_nodes = []

    print("Welcome to the Akinator Game!")
    print("I am your virtual Akinator, ready to guess the character you are thinking of.")
    print("Answer a series of yes/no questions, and I'll do my best to identify your character.")
    print("If at any point you change your mind or want to go back, just type 'back'.")
    print("Let's begin!\n")

    while True:
        if model.tree_.children_left[node] == model.tree_.children_right[node]:  # If leaf node
            prediction_prob = model.tree_.value[node].max() / model.tree_.value[node].sum()
            prediction = model.classes_[model.tree_.value[node].argmax()]
            cumulative_confidence *= prediction_prob
            if cumulative_confidence >= 100.0:
                character_info = characters_df[characters_df['character_name'] == prediction]
                character_description = character_info['description'].values[0]
                character_birthday = character_info['birthday'].values[0]
                character_realease = character_info['release_date'].values[0]
                character_affiliation = character_info['affiliation'].values[0]
                character_ascension = character_info['ascension_specialty'].values[0]
                print(f"I am 100% confident that the character is {prediction}.")
                print(f"{prediction}'s Description: {character_description}")
                print(f"{prediction}'s Birthday: {character_birthday}")
                print(f"{prediction}'s Release Date: {character_realease}")
                print(f"{prediction}'s Affiliation: {character_affiliation}")
                print(f"{prediction}'s Ascension: {character_ascension}")
                print("I am 100% confident. Character identified!")
            else:
                print(f"Now {100 - cumulative_confidence:.2f}% confident. Moving to the next question.")
            break

        feature_index = model.tree_.feature[node]
        feature_name = features[feature_index]
        answer = ask_question(feature_name)

        if answer.lower() == 'back':
            if previous_nodes:
                node = previous_nodes.pop()
                cumulative_confidence = 100.0
                print("Going back to the previous question.")
            else:
                print("Cannot go back further. Already at the beginning.")
                continue
                
        elif answer.lower() == 'no':
            cumulative_confidence *= (1 - model.tree_.value[node].max() / model.tree_.value[node].sum())
            previous_nodes.append(node)
            node = model.tree_.children_left[node]
        elif answer.lower() == 'yes':
            cumulative_confidence *= (1 - model.tree_.value[node].max() / model.tree_.value[node].sum())
            previous_nodes.append(node)
            node = model.tree_.children_right[node]
        else:
            print("Error: Invalid answer. Please enter 'yes', 'no', or 'back'. Try again.")
            continue
            
        remaining_characters = model.tree_.value[node].sum()
        if remaining_characters == 1:
            cumulative_confidence = 100.0

        if cumulative_confidence < 100.0:
            print(f"Now {100 - cumulative_confidence:.2f}% confident. Moving to the next question.")

akinator_game(clf, features, df)

Welcome to the Akinator Game!
I am your virtual Akinator, ready to guess the character you are thinking of.
Answer a series of yes/no questions, and I'll do my best to identify your character.
If at any point you change your mind or want to go back, just type 'back'.
Let's begin!

Does the character have has geo vision? (yes/no/back): yes
Now 1.32% confident. Moving to the next question.
Does the character have is female? (yes/no/back): no
Now 15.41% confident. Moving to the next question.
Does the character have is archon? (yes/no/back): yes
I am 100% confident that the character is Zhongli.
Zhongli's Description: nan
Zhongli's Birthday: 31-Dec
Zhongli's Release Date: 12.01.2020
Zhongli's Affiliation: Liyue Harbor
Zhongli's Ascension: Cor Lapis
I am 100% confident. Character identified!
